In [6]:
# Load XLSX and create dropdown of columns
import sys
import numpy as np
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
from IPython.display import clear_output
import IPython.display as dsply
import ipywidgets as widgets
# Jupyter will always display the results of the last line of code we want to return a "box" with all widgets
# we want displayed
from ipywidgets import Box

import OpticsFunctions
import importlib

# this re-loads every exectution (for now) so we can make code changes
importlib.reload(OpticsFunctions)

input_file_path = "INPUT/Table5B_For_Search.xlsx"
df = pd.read_excel(input_file_path, engine="openpyxl")

search_type = "string"
search_min_val = 0
search_max_val = 0

search_column = ""
search_value = ""

search_description = ""

filtered_df = pd.DataFrame()

debugEnabled = False

def set_search_params(Search):
    global search_column
    global search_type
    global search_min_val
    global search_max_val

    search_column = Search
    if debugEnabled: 
        print("set_search_params() -> search_column has been set to " + search_column)
    
    # collect data types
    data_type_present_in_target = df[search_column].dtype
    # print(f"Search column contains the following data types: {data_type_present_in_target}")

    stringSearchBox.layout.display = 'none'
    minNumSearchBox.layout.display = 'none'
    maxNumSearchBox.layout.display = 'none'

    if (data_type_present_in_target == 'object'):
        search_type = "string"
        stringSearchBox.layout.display = ''
        #clean NaN values.  since it's a string column, just use "No Dada"
        df[search_column].fillna("No Data", inplace = True)
    elif (data_type_present_in_target == 'float64'):
        search_type = "number"
        minNumSearchBox.layout.display = ''
        maxNumSearchBox.layout.display = ''
        
        search_min_val = df[search_column].min()
        search_max_val = df[search_column].max()
        minNumSearchBox.Value = search_min_val
        maxNumSearchBox.Value = search_max_val
        # print("min value:" + str(search_min_val))
        # print("max value:" + str(search_max_val))

    if debugEnabled: 
        print("set_search_params() -> search_type has been set to " + search_type)
    
def perform_search():
    global search_type
    global search_column
    global search_value
    global search_min_val
    global search_max_val
    global filtered_df
    global search_description

    # print(f"Worksheet contains the following data types: {data_frame.dtypes}")
    # print(search_column)
    # print(search_value)
    
    
    try:
        if (search_type == "string"):
            search_description = "search_column=>" + search_column + ", search_type=>" + search_type + ", search_value=>" + search_value
            
            filtered_df  = df[df[search_column].str.contains(search_value, case=False)]
        else:
            search_description = "search_column=>" + search_column + ", search_type=>" + search_type + ", min=>" + search_min_val + ", max=>" + search_max_val
    
            # print(f"Unable to run text query: {e}")
            # print("reverting to basic query that will work for numbers")
    
            # generate our query (if column names have whitespace we must wrap in back ticks)
            # query_to_run = f"`{search_column}` == {search_value}"
            # query_to_run = f"`{search_column}` >= {str(search_min_val)}" # and `{search_column}` <= {str(search_max_val)}"
    
            numeric_minimum = float(search_min_val)
            numeric_maximum = float(search_max_val)
            filtered_df = df[df[search_column].between(numeric_minimum, numeric_maximum)]
            
            # print(f"running query -> {query_to_run}")
    
            # TODO: might be better to do all this with inplace=True but would require reworking
            # result_df = df.query(query_to_run, inplace=False)
            
        # return result_df
    except:
        # Do nothing for now
        # print("unable to run query")
        print("")


# Make the options dynamic to the columns of the data table
dataColumns = list(df.columns)
# TODO: add ability to search all columns
# dataColumns.insert(0, "< Any Column >")

 #def columnChosen(Search):
#   set_search_params(Search)
    
def stringSearchHandler(Containing): 
    # set_search_params(c=Search, v=Value)
    global search_value
    search_value = Containing
    if debugEnabled: 
        print("stringSearchHandler() -> string value found ->" + Value);
    # perform_search()

def numberSearchHandlerMinimum(Minimum): 
    global search_min_val
    if debugEnabled: 
        print("numberSearchHandlerMinimum() -> numeric value found ->" + str(Minimum));
    search_min_val = Minimum
    # perform_search()
    
def numberSearchHandlerMaximum(Maximum): 
    global search_max_val
    if debugEnabled: 
        print("numberSearchHandlerMaximum() -> numeric value found ->" + str(Maximum));
    search_max_val = Maximum
    # perform_search()
    
def h(p, q):
    return (p, q)

stringSearchBox = interactive(stringSearchHandler, Containing="")
minNumSearchBox = interactive(numberSearchHandlerMinimum, Minimum=(str(search_min_val)))
maxNumSearchBox = interactive(numberSearchHandlerMaximum, Maximum=(str(search_max_val)))

def toggleDebug(Display_Debug_Messages):
    global debugEnabled
    debugEnabled = Display_Debug_Messages
    
debugCheckBox = interactive(toggleDebug, Display_Debug_Messages=debugEnabled)
searchColumnDropdown = interactive(set_search_params, Search=dataColumns)

display(debugCheckBox, searchColumnDropdown)

# comment the line out below to enable debug
debugCheckBox.layout.display = "none"

# we get a weird output without a print here
# TODO: figure out what's going on
print("")

interactive(children=(Checkbox(value=False, description='Display_Debug_Messages'), Output()), _dom_classes=('w…

interactive(children=(Dropdown(description='Search', options=('Material', 'Substrate', 'Fabrication', 'Thickne…

In [2]:
# when this cell is executed it will render proper search controls for the selected column
# either a text box for a string column or min/max range boxes for a numeric column

dsply.display(stringSearchBox, minNumSearchBox, maxNumSearchBox)



    

interactive(children=(Text(value='', description='Value'), Output()), layout=Layout(display=''), _dom_classes=…

interactive(children=(Text(value='0', description='Minimum'), Output()), layout=Layout(display='none'), _dom_c…

interactive(children=(Text(value='0', description='Maximum'), Output()), layout=Layout(display='none'), _dom_c…

In [3]:
def click_runSearch(b):
    perform_search()
    
    with runSearchOutput:
        clear_output()
        if debugEnabled: 
            print(search_description)

        display(filtered_df)

runSearchOutput = widgets.Output()
runSearchButton = widgets.Button(description="Run Search")
runSearchButton.on_click(click_runSearch)

dsply.display(runSearchButton, runSearchOutput)

#This just makes the data frame display initially (with no filter)
click_runSearch(runSearchButton)

Button(description='Run Search', style=ButtonStyle())

Output()

In [4]:
def click_run_export(b):
    
    with runExportOutput:
        clear_output()
        OpticsFunctions.export_to_csv(filtered_df)

runExportOutput = widgets.Output()
runExportButton = widgets.Button(description="Export Results")
runExportButton.on_click(click_run_export)

dsply.display(runExportButton, runExportOutput)


Button(description='Export Results', style=ButtonStyle())

Output()